In [1]:
from pyModbusTCP.client import ModbusClient
import threading
import time
import configparser
import logging

import warnings
warnings.filterwarnings('ignore')
# Logging and Configuration
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

class MoistureSensor:
    def __init__(self, config_path='config.ini'):
        self.config = configparser.ConfigParser()
        self.config.read(config_path)
        sensor_config = self.config['MoistureSensor']
        
        self.host = sensor_config.get('host')
        self.port = sensor_config.getint('port')
        self.address = sensor_config.getint('address')
        self.num_registers = sensor_config.getint('num_registers')
        self.sensorid = sensor_config.get('sensorid')
        
        self.client = ModbusClient(host=self.host, port=self.port, auto_open=True)
        
    def read_data(self):
        try:
            data = self.client.read_holding_registers(self.address, self.num_registers)
            if data and len(data) > 2:  # ตรวจสอบว่ามีข้อมูลและมีความยาวมากกว่า 2
                return data[2]  # Return ค่าที่ index ที่ 2
            else:
                return None  # หรือค่าที่เหมาะสมสำหรับสถานการณ์ที่ไม่มีข้อมูลหรือข้อมูลไม่เพียงพอ
        except Exception as e:
            logging.error(f"Error reading data from moisture sensor: {e}")
            return None

    def close_connection(self):
        self.client.close()

    def continuous_read(self, interval=1):
        try:
            """อ่านค่าเซ็นเซอร์แบบต่อเนื่องทุกๆ interval วินาที."""
            def read_loop():
                while True:
                    data = self.read_data()
                    print(data)  # หรือจัดการข้อมูลที่นี่
                    time.sleep(interval)
            
            thread = threading.Thread(target=read_loop)
            thread.daemon = True  # ทำให้ thread สิ้นสุดเมื่อโปรแกรมหลักสิ้นสุด
            thread.start()
            
        except Exception as e:
            logging.error(f"Error opening camera: {e}")

# ตัวอย่างการใช้งาน
sensor = MoistureSensor()
sensor.continuous_read(interval=1)  # อ่านค่าทุก 5 วินาที


1672
52
52
39
52
52
45
52
45
52
52
52
39
39
45
52
52
52
52
52
52
58
52
45
52
52
52
45
52
52
58
45
52
45
52
65
58
45
52
45
39
58
45
65
52
52
52
52
39
52
45
58
52
52
45
52
45
52
52
45
52
45
52
52
52
58
52
58
58
52
45
45
45
45
39
39
52
45
39
52


In [1]:
from pyModbusTCP.client import ModbusClient
import threading
import time
import configparser
import logging
import warnings
from paho.mqtt import client as mqtt_client

warnings.filterwarnings('ignore')
# Logging and Configuration
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

class MoistureSensor:
    def __init__(self, config_path='config.ini'):
        self.config = configparser.ConfigParser()
        self.config.read(config_path)
        sensor_config = self.config['MoistureSensor']

        self.host = sensor_config.get('host')
        self.port = sensor_config.getint('port')
        self.address = sensor_config.getint('address')
        self.num_registers = sensor_config.getint('num_registers')
        self.sensorId = sensor_config.get('sensorId')

        self.client = ModbusClient(host=self.host, port=self.port, auto_open=True)

        # MQTT Configuration
        mqtt_config = self.config['MQTT']
        self.mqtt_broker = mqtt_config.get('broker')
        self.mqtt_port = mqtt_config.getint('port')
        self.mqtt_topic = mqtt_config.get('topic')
        self.mqtt_client_id = f'python-mqtt-{self.sensorId}'

    def mqtt_connect(self):
        def on_connect(client, userdata, flags, rc):
            if rc == 0:
                logging.info("Connected to MQTT Broker!")
            else:
                logging.error("Failed to connect, return code %d\n", rc)

        self.mqtt_client = mqtt_client.Client(self.mqtt_client_id)
        self.mqtt_client.on_connect = on_connect
        self.mqtt_client.connect(self.mqtt_broker, self.mqtt_port)
        self.mqtt_client.loop_start()

    def read_data_and_publish(self):
        try:
            while True:
                data = self.client.read_holding_registers(self.address, self.num_registers)
                if data and data[6] == 1:
                    self.mqtt_client.publish(self.mqtt_topic, str(data[0]))
                elif data and data[6] == 0:
                    break  # หยุดการส่งข้อมูล
                time.sleep(1)  # รอ 1 วินาทีก่อนอ่านข้อมูลครั้งถัดไป
        except Exception as e:
            logging.error(f"Error reading data from moisture sensor: {e}")

    def start(self):
        self.mqtt_connect()
        read_thread = threading.Thread(target=self.read_data_and_publish)
        read_thread.daemon = True
        read_thread.start()

if __name__ == "__main__":
    sensor = MoistureSensor()
    sensor.start()


ValueError: Unsupported callback API version: version 2.0 added a callback_api_version, see migrations.md for details